# Test Code for the Galactic Plane Footprint Metric

Code adapted from an example by Lynne Jones

In [134]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import rubin_sim.maf as maf

Load the baseline v2.0 as a test case OpSim:

In [135]:
from rubin_sim.data import get_baseline

opsim_fname = get_baseline()
print(opsim_fname)

runName = os.path.split(opsim_fname)[-1].replace('.db', '')
print(runName)
opsim_db = maf.OpsimDatabase(opsim_fname)

/Users/rstreet1/rubin_sim_data/sim_baseline/baseline_v2.0_10yrs.db
baseline_v2.0_10yrs


Taking an example sky location within the Galactic Bulge:

In [136]:
test_ra = (17.0 + 57.0/60.0 + 34.0/3600.0)*15.0
test_dec = (29.0 + 13.0/60.0 + 15.0/3600.0)*-1.0
test_slicer = maf.UserPointsSlicer(test_ra, test_dec)

In [150]:
import numpy as np
import healpy as hp
from astropy import units as u
from astropy_healpix import HEALPix
from astropy.coordinates import Galactic, TETE, SkyCoord

class galPlaneFootprintMetric(maf.BaseMetric):
    """Metric to evaluate the survey overlap with desired regions in the Galactic Plane
    and Magellanic Clouds, by referencing the pre-computed priority maps provided.

    Parameters
    ----------
    fieldRA : float, RA in degrees of a given pointing
    fieldDec : float, Dec in degrees of a given pointing
    filter : str, filter bandpass used for a given observation
    """
    
    def __init__(self, cols=['fieldRA','fieldDec','filter','fiveSigmaDepth'],
                       metricName='GalPlaneFootprintMetric',
                       **kwargs):
        """Kwargs must contain:
        filters  list Filterset over which to compute the metric
        """

        self.ra_col = 'fieldRA'
        self.dec_col = 'fieldDec'
        self.filterCol = 'filter'
        self.m5Col = 'fiveSigmaDepth'
        self.filters = ['u','g', 'r', 'i', 'z', 'y']
        self.magCuts = {'u': 22.7, 'g': 24.1, 'r': 23.7, 'i': 23.1, 'z': 22.2, 'y': 21.4}
        cwd = os.getcwd()
        self.MAP_DIR = os.path.join(cwd,'../../data/galPlane_priority_maps')
        self.MAP_FILE_ROOT_NAME = 'GalPlane_priority_map'
        self.load_maps()

        super().__init__(col=cols, metricName=metricName)

    def load_maps(self):
        self.NSIDE = 64
        self.NPIX = hp.nside2npix(self.NSIDE)
        self.ideal_combined_map = np.zeros(self.NPIX)
        for f in self.filters:
            fmap = hp.read_map(os.path.join(self.MAP_DIR,self.MAP_FILE_ROOT_NAME+'_'+str(f)+'.fits'))
            setattr(self, 'map_'+str(f), fmap)
            self.ideal_combined_map += fmap
        
    def run(self, dataSlice, slicePoint=None):

        combined_map = np.zeros(self.NPIX)
        
        for f in self.filters:
            idx1 = np.where(dataSlice[self.filterCol] == f)[0]
            idx2 = np.where(dataSlice[self.m5Col] >= self.magCuts[f])[0]
            match = list(set(idx1).intersection(set(idx2)))
            
            coords_icrs = SkyCoord(dataSlice[self.ra_col][match], dataSlice[self.dec_col][match], frame='icrs', unit=(u.deg, u.deg))
            coords_gal = coords_icrs.transform_to(Galactic()) 
            ahp = HEALPix(nside=self.NSIDE, order='ring', frame=TETE())
            pixels = ahp.skycoord_to_healpix(coords_gal)
            
            weighted_map = getattr(self, 'map_'+str(f))
            combined_map[pixels] += weighted_map[pixels]
            
        metric_value = combined_map.sum()
        
        # Normalize by full weighted map summed over all filters and pixels:
        metric_value /= self.ideal_combined_map[pixels].sum()

        return metric_value

In [151]:
mymetric = galPlaneFootprintMetric()

Constraints, if any:

In [152]:
sqlconstraint = None

Construct the metric bundle from the metric, the slicer and the constraints, and add it to a bundle group:

In [153]:
bundle = maf.MetricBundle(mymetric, test_slicer, sqlconstraint, runName=runName)
g = maf.MetricBundleGroup({'test_metric': bundle}, opsim_db, outDir='test', resultsDb=None)

Calculate the metric:

In [154]:
g.runAll()

Querying database observations with no constraint for columns ['rotSkyPos', 'fieldRA', 'filter', 'fiveSigmaDepth', 'fieldDec'].
Found 2086980 visits
Running:  ['test_metric']
[  4  11  14  19  35  64  75  89 102 162 199 200 229 242 265 267 321 331
 338 431 434 446 472 481 580 621 644 672 708 732 808 815]
[  0   1   2   3   4   5   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  25  26  28  29  30  31  32  33  35  36  37  38  39
  40  41  44  45  46  47  48  50  52  53  54  55  56  57  58  59  60  61
  62  63  64  65  66  67  69  70  72  74  75  76  77  79  80  81  83  84
  85  86  87  88  89  90  91  92  93  94  97  98  99 100 102 103 104 105
 106 108 109 110 111 112 114 115 116 118 119 120 121 122 123 124 125 126
 127 129 130 132 133 134 135 136 137 138 139 140 142 143 145 146 147 148
 149 150 151 152 153 154 155 156 157 158 161 162 165 167 168 170 171 172
 173 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191
 192 193 194 195 196 197 198 199 200 

In [155]:
bundle.metricValues

masked_array(data=[0.34027237354085604],
             mask=[False],
       fill_value=-666.0)